# Génération de graphes

Ce notebook sert à générer des graphes aléatoires de taille et probabilité donnée, et fournit également le début d'un algorithme pouvant générer un arbre en Neo4J, de taille et nombre de fils maximum donnés

In [ ]:
from neo4j import GraphDatabase

In [ ]:
import numpy as np
import pickle as pkl
from scipy.sparse import csgraph, csr_matrix
from random import random, randrange, seed

#### Base Euclid

In [ ]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j","euclid"))

#### Base Test

In [ ]:
driver = GraphDatabase.driver("bolt://localhost:11003", auth=("neo4j","0"))

----------

In [ ]:
session = driver.session()

---------

### Génération de graphes aléatoires

In [ ]:
def generate_graph_script(nb_noeuds, rand_th=0.2):
    """
    Génère le script de génération du graphe aléatoire
    """
    with open(f"random_graph{nb_noeuds}.cypher", "w") as out:
        #Création des noeuds
        for i in range(1,nb_noeuds+1):
            out.write(f'CREATE (n{i}:Node{{name:"n{i}"}})\n')
            
        #Création des relations
        for n1 in range(1,nb_noeuds+1):
            for n2 in range(n1+1, nb_noeuds+1):
                r = random()
                poids = 0
                if r <= rand_th:
                    poids = round(random(),3)
                    out.write(f'CREATE (n{n1})-[:LINKED_TO{{poids:{poids}}}]->(n{n2})\n')
                #print(f'n{n1}', f'n{n2}', r, poids)
        out.write(";")

In [ ]:
def generate_graph(nb_noeuds, rand_th=0.2):
    """
    Génère le graphe aléatoire directement dans Neo4J
    """
    #Suppression des noeuds et relations déjà présentes dans la base
    session.run("MATCH (n) DETACH DELETE n")
    
    script = ""

    #Création des noeuds
    for i in range(1,nb_noeuds+1):
        script += f'CREATE (n{i}:Node{{name:"n{i}"}})\n'

    #Création des relations
    for n1 in range(1,nb_noeuds+1):
        for n2 in range(n1+1, nb_noeuds+1):
            r = random()
            poids = 0
            if r <= rand_th:
                poids = round(random(),3)
                script += f'CREATE (n{n1})-[:LINKED_TO{{poids:{poids}}}]->(n{n2})\n'

    script += ";"

    #Exécution du script au lieu d'une simple sauvegarde
    session.run(script)


In [ ]:
def generate_adj(nb_noeuds,rand_th=0.2):   
    """
    Génère la matrice d'adjacence d'un graphe aléatoire
    sans avoir à générer le graphe lui-même
    """
    #Initialisation de la matrice
    Ax = np.zeros((nb_noeuds, nb_noeuds))
    
    #Création des relations
    for n1 in range(nb_noeuds):
        for n2 in range(n1+1, nb_noeuds):
            r = random()
            poids = 0
            if r <= rand_th:
                poids = round(random(),3)
                
                #ici, Relation non orientée
                Ax[n1,n2] = poids
                Ax[n2,n1] = poids
                
    return Ax

-------

### Génération d'arbres

In [ ]:
def generate_arbre(nb_noeuds, max_fils, nb_attr):
    """
    Génère un graphe sous forme d'arbre directement dans Neo4J (profondeur pas implémentée)
    """
    
    #Suppression des noeuds et relations déjà présents sur la base
    session.run("MATCH (n) DETACH DELETE n")
    
    script = ""
    
    #Liste de tous les noeuds
    noeuds = np.arange(1,nb_noeuds)
    
    #Le noeud 0 est le premier noeud de l'arborescence
    lvl = [0]
    
    fils = dict()
    
    #Création des noeuds avec un nombre donné de paramètres aléatoires
    for i in range(nb_noeuds):
        random_attrs = [f"x{i_attr}:{random()}" for i_attr in range(nb_attr)]
        script += f"CREATE (n{i}:Node{{name:'n{i}',{','.join(random_attrs)}}})\n"

    while len(noeuds) != 0:
        #print(f"NB NOEUDS: {len(noeuds)}\n")
        for node in lvl:
            #print(f'FILS DE {node}')
            
            #On choisit entre 1 et max_fils noeuds comme fils de node
            sample = np.random.choice(noeuds,randrange(1,1+min(len(noeuds), max_fils)),replace=False)
            #print(sample)
            
            #On enlève les noeuds du sample de la liste de noeuds
            noeuds = noeuds[np.isin(noeuds, sample, invert=True)]
            #print(noeuds)
            
            
            for val in sample:
                script += f"CREATE (n{val})-[:CHILD_OF]->(n{node})\n"

            lvl = sample
            fils[node] = sample

            if len(noeuds) == 0:
                break
    script += ";"
    
    session.run(script)

In [ ]:
def generate_arbre_script(nb_noeuds, max_fils, nb_attr):
    """
    Crée le script pour la génération d'un arbre en Neo4J (profondeur pas implémentée)
    """
    
    #Liste des noeuds
    noeuds = np.arange(1,nb_noeuds)
    
    #Le noeud 0 est le premier noeud de l'arborescence
    lvl = [0]
    
    fils = dict()
    
    with open(f'treeGraph{nb_noeuds}_{max_fils}.cypher','w') as out:
        #Création des noeuds avec un nombre donné de paramètres aléatoires
        for i in range(nb_noeuds):
            random_attrs = [f"x{i_attr}:{random()}" for i_attr in range(nb_attr)]
            out.write(f"CREATE (n{i}:Node{{name:'n{i}',{','.join(random_attrs)}}})\n")
            
        while len(noeuds) != 0:
            #print(f"NB NOEUDS: {len(noeuds)}\n")
            for node in lvl:
                #print(f'FILS DE {node}')
                
                #On choisit entre 1 et max_fils noeuds comme fils de node
                sample = np.random.choice(noeuds,randrange(1,1+min(len(noeuds), max_fils)),replace=False)
                #print(sample)
                
                #On enlève les noeuds du sample de la liste de noeuds
                noeuds = noeuds[np.isin(noeuds, sample, invert=True)]
                #print(noeuds)
                
                for val in sample:
                    out.write(f"CREATE (n{val})-[:CHILD_OF]->(n{node})\n") 

                lvl = sample
                fils[node] = sample

                if len(noeuds) == 0:
                    break
        out.write(";")

In [ ]:
def generate_arbre_adj(nb_noeuds, max_fils): 
    """
    Génère la matrice d'adjacence de l'arbre (profondeur pas implémentée)
    """
    At = np.zeros((nb_noeuds, nb_noeuds))
    noeuds = np.arange(1,nb_noeuds)
    
    lvl = [0]
    
    fils = dict()
    
    while len(noeuds) != 0:
        for node in lvl:
            sample = np.random.choice(noeuds,randrange(1,1+min(len(noeuds), max_fils)),replace=False)
            noeuds = noeuds[np.isin(noeuds, sample, invert=True)]

            #Les relations entre les fils et leur père sont mises à 1
            At[node][sample] = 1    
            
            lvl = sample
            fils[node] = sample
            
            if len(noeuds) == 0:
                break
    
    return csr_matrix(At+At.T)

### Test de performance

In [ ]:
from time import time

start = time()
generate_graph(200)
print(f"Execution time: {time()-start:.2f}s")

In [ ]:
start = time()
generate_adj(200)
print(f"Execution time: {time()-start:.2f}s")

In [ ]:
start = time()
generate_arbre_adj(200,5)
print(f"Execution time: {time()-start:.2f}s")

Générer simplement la matrice d'adjacence logiquement énormément plus rapide

-----------

## Calculs

In [ ]:
def remove_names_NS(node_names, namespaces):
    """
    Fonction pour retirer dans la requête les noeuds du namespace
    qui commencent par les chaînes contenues dans la liste
    """
    out_cond = ""
    
    if namespaces:
        out_cond = 'NOT ('
        out_cond += ") AND NOT (".join(" OR ".join(f'{node_name}.fullname =~ "{ns}.*"' for ns in namespaces) for node_name in node_names)
        out_cond += ')'
        
    return out_cond

 Namespaces à enlever selon la version :
 
* V1 -> Aucune restriction
* V2 -> Pas de namespace lié à la DPD
* V3 -> V2 + pas de namespace lié à PRO
* V4 -> V3 + pas de namespace lié à INTERFACES

In [ ]:
nodeType, relType, version, use_weights = "Type", "USE_TYPE", "V4", True

In [ ]:
str_weight = 'W' if use_weights and relType in  {'USE_NS', 'USE_TYPE'} else ''
graph_suffix = f"{relType}{version}{str_weight}"

In [ ]:
type_same_ns = False

cond = ""

if version == "V1":
    remove_ns = []
elif version == "V2":
    remove_ns = ['dpd']
elif version == "V3":
    remove_ns = ['dpd','pro']
elif version == "V4":
    remove_ns = ['dpd','pro','interfaces']
    
if version not in {"V0","V1"}:
    if nodeType == 'Type':
        nodeNames = ['ns'] if type_same_ns else ['ns','ns2'] 
    else:
        nodeNames = ['n1','n2']
    
    cond = f"WHERE {remove_names_NS(nodeNames,remove_ns)}"

In [ ]:
def query_start(d="",same_ns=True):
    """
    Génère le début de la requête Cypher
    
    d: string pour donner la direction de la relation
    same_ns: indique si les noeuds doivent faire partie du même namespace
    
    q: le début de requête
    """
    q = f'(n1:{nodeType}){"<-" if d == "l" else "-"}[r:{relType}]{"->" if d == "r" else "-"}(n2:{nodeType})'  
    
    if nodeType == "Type":
        q = f"(ns:Namespace)<-[:DECLARED_IN]-{q}-[:DECLARED_IN]->({'ns' if same_ns else 'ns2:Namespace'})"
        
    q = f'MATCH {q} {cond}'
    
    return q

In [ ]:
query_start(same_ns=type_same_ns)

### Silhouette et inertie

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
if nodeType == "Type":
    #Certains types n'ont pas de relations associées donc on ne peut pas juste
    #faire un COUNT sur les types avec la relation
    q_nbnoeuds_total = session.run(f"MATCH (n1:{nodeType})-[:DECLARED_IN]->(n:Namespace) WHERE {remove_names_NS(['n'],remove_ns)} RETURN COUNT(DISTINCT n1) AS nb")
else:
    q_nbnoeuds_total = session.run(f"{query_start(same_ns=type_same_ns)} RETURN COUNT(DISTINCT n1) AS nb")
    
q_nbnoeuds = session.run(f"{query_start(same_ns=type_same_ns)} RETURN COUNT(DISTINCT n1) AS nb")
q_nbaretes = session.run(f"{query_start(same_ns=type_same_ns)} RETURN COUNT(DISTINCT r) AS nb")

#### Résultats de la requête

In [ ]:
for res in q_nbnoeuds_total: nbnoeudstotal = res["nb"]
for res in q_nbnoeuds: nbnoeuds = res["nb"]
for res in q_nbaretes: nbaretes = res["nb"]

In [ ]:
n = nbnoeuds

#Coefficient de connexité du graphe
r = (2*nbaretes)/(nbnoeudstotal*(nbnoeudstotal-1))

In [ ]:
print(nbnoeudstotal, nbnoeuds, nbaretes, r)

#### Calcul des mesures pour les graphes aléatoires 

In [ ]:
nb_graphes = 50
nb_iter = 2
max_k = min(n,81)

inertias = []
silhs = []
vpRand = []

for nb_g in range(nb_graphes):
    if (nb_g+1) % 5 == 0 or (nb_g+1) == nb_graphes: 
        print("="*30)
        print(f"GRAPHE {nb_g+1}/{nb_graphes}")
        print("="*30)
        
    #Calcul des valeurs propres pour le partitionnement
    vp_norm, vectp_norm = np.linalg.eigh(csgraph.laplacian(generate_adj(n), normed=True))
    
    #La version utilisant le coefficient n'a donné de bons résultats,
    #elle n'est donc plus utilisée ici
    #vp_norm, vectp_norm = np.linalg.eigh(csgraph.laplacian(generate_adj(n,r), normed=True))
    
    vpRand.append(vp_norm)
    
    #Moyenne de n itérations de calculs différents de KMeans de 2 à 80 max
    for i in range(nb_iter):
        print(f"ITERATION {i+1}/{nb_iter}")
        inertia = []
        silh = []
        for k in range(2,max_k):
            if k%10 == 0 or k == max_k-1 : print(f"{k}/{max_k-1}")
            kmeans = KMeans(n_clusters=k).fit(vectp_norm[:,:k])
            inertia.append(kmeans.inertia_)
            silh.append(silhouette_score(vectp_norm[:,:k], kmeans.labels_))
        inertias.append(inertia)
        silhs.append(silh)
    print("-"*20)

avg_inertia = np.array(inertias).mean(0)
avg_silh = np.array(silhs).mean(0)
avg_vp = np.array(vpRand).mean(0)

#### Sauvegarde des résultats

In [ ]:
pkl.dump(avg_inertia, open(f"randInertia{graph_suffix}.pkl","wb"))
pkl.dump(avg_silh, open(f"randSilh{graph_suffix}.pkl","wb"))
pkl.dump(avg_vp, open(f"randVp{graph_suffix}.pkl","wb"))

-----

### Affichage des résultats

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,10)

#### Inertie

In [ ]:
plt.scatter(range(2,avg_inertia.shape[0]+2), avg_inertia)

#### Silhouette

In [ ]:
plt.scatter(range(2,avg_silh.shape[0]+2), avg_silh)